In [1]:
#importing packages
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
my_dict_rev = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 'a', '11': 'b', '12': 'c', '13': 'd', '14': 'e', '15': 'f', '16': 'g', '17': 'h', '18': 'i', '19': 'k', '20': 'l', '21': 'm', '22': 'n', '23': 'o', '24': 'p', '25': 'q', '26': 'r', '27': 's', '28': 't', '29': 'u', '30': 'v', '31': 'w', '32': 'x', '33': 'y','34':'z'}

my_dict = {str(value) : int(key) for key,value in my_dict_rev.items()}

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cols= ['x'+str(x) for x in range(0,63)]
cols.append('y')

#df = pd.DataFrame(columns=cols)
df = pd.read_csv(r"C:\Users\Srinivas\Desktop\SignLanguagePrediction_1\finalhands.csv")

df = df.iloc[:,1:]

df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x54,x55,x56,x57,x58,x59,x60,x61,x62,y
0,0.721064,0.680146,5.305915e-07,0.665897,0.633042,-0.033922,0.636554,0.541967,-0.057276,0.631913,...,0.784999,0.577582,-0.118721,0.785405,0.549660,-0.123090,0.787616,0.520700,-0.125752,10.0
1,0.742452,0.729304,-6.583956e-07,0.665038,0.665285,-0.029834,0.626336,0.552821,-0.037984,0.627644,...,0.816697,0.498060,-0.030620,0.796605,0.567541,-0.017358,0.783598,0.604849,0.001076,10.0
2,0.742435,0.726822,-6.720254e-07,0.664789,0.657637,-0.026775,0.627332,0.545786,-0.033937,0.629642,...,0.815308,0.487969,-0.027896,0.794712,0.555203,-0.012999,0.786008,0.593632,0.006465,10.0
3,0.727032,0.668081,3.797623e-07,0.669312,0.653536,-0.041104,0.632739,0.584202,-0.068620,0.630427,...,0.777863,0.528485,-0.112833,0.775756,0.558075,-0.106270,0.780093,0.584329,-0.094985,10.0
4,0.741162,0.722990,-6.227788e-07,0.666857,0.656922,-0.030431,0.625908,0.545657,-0.039670,0.623722,...,0.813357,0.486171,-0.028489,0.795381,0.552095,-0.017682,0.782524,0.592645,-0.000487,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,0.258570,0.773129,-2.155658e-07,0.312682,0.735135,-0.018480,0.341341,0.666464,-0.025035,0.330383,...,0.225152,0.579440,-0.031733,0.237103,0.623130,-0.024732,0.241942,0.652316,-0.013007,28.0
2796,0.256157,0.781323,-8.506021e-08,0.308432,0.739674,-0.022460,0.335501,0.674863,-0.032760,0.324521,...,0.223522,0.588345,-0.029544,0.237698,0.625718,-0.025106,0.244374,0.653657,-0.015743,28.0
2797,0.262534,0.773058,-1.691849e-07,0.311931,0.737734,-0.014090,0.340429,0.670538,-0.017554,0.327528,...,0.227385,0.587791,-0.025171,0.237545,0.627344,-0.019149,0.242846,0.656623,-0.008777,28.0
2798,0.270180,0.772911,-2.010464e-07,0.323270,0.734255,-0.017303,0.350198,0.669251,-0.023999,0.336021,...,0.240997,0.589664,-0.028533,0.250931,0.629636,-0.022280,0.254729,0.657516,-0.012083,28.0


In [19]:
# for creating dataset

cap = cv2.VideoCapture(0)
icount = [0,0,0,0,0]
#putdata = []

with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence = 0.5) as hands:
    
    while cap.isOpened():
        ret , frame = cap.read()
        
        putcurrent = []
        
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        image = cv2.flip(image,1)
        
        image.flags.writeable = False
        result = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        #print(result)
        
        if result.multi_hand_landmarks:
            for num,hand in enumerate(result.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(0,0,256) , thickness=2,circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(0,0,0) , thickness=2,circle_radius=2))
        
        #inserting into dataframe
        pressed = cv2.waitKey(10)
        if pressed != -1 and pressed != ord('z'):
                for i in result.multi_hand_landmarks[0].landmark:
                    putcurrent.append(i.x)
                    putcurrent.append(i.y)
                    putcurrent.append(i.z)
                
                putcurrent.append(my_dict[chr(pressed)])
                df.loc[len(df)] = putcurrent
            
        elif pressed  == ord('z'):
                break
        cv2.imshow('Pleiades was here' , image)
        
cap.release()
cv2.destroyAllWindows()

In [20]:
#df
df.shape

(3467, 64)

In [21]:
df.to_csv(r'C:\Users\Srinivas\Desktop\SignLanguagePrediction_1\finalhands.csv')

In [35]:
DF = pd.read_csv(r'C:\Users\Srinivas\Desktop\SignLanguagePrediction_1\finalhands.csv')
#DF

In [36]:
from sklearn.model_selection import train_test_split

DF = pd.read_csv(r'C:\Users\Srinivas\Desktop\SignLanguagePrediction_1\finalhands.csv')
X = DF.drop(['Unnamed: 0','y'],axis=1)
y = DF['y']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=13)

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

X_train.shape , y_train.shape

((1960, 63), (1960,))

In [37]:
xt1 = X_train[:]
yt1 = y_train[:]
xt1.shape,yt1.shape

((1960, 63), (1960,))

In [38]:
# ANN

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(63,)))
model.add(tf.keras.layers.Dense(63,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(34,activation='softmax'))


In [39]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [40]:
model.fit(xt1,yt1,epochs=2000)

Epoch 1/2000
62/62 [==============================] - 3s 7ms/step - loss: 3.3411 - accuracy: 0.0592
Epoch 2/2000
62/62 [==============================] - 0s 5ms/step - loss: 3.0990 - accuracy: 0.0816
Epoch 3/2000
62/62 [==============================] - 0s 5ms/step - loss: 2.9450 - accuracy: 0.1219
Epoch 4/2000
62/62 [==============================] - 0s 7ms/step - loss: 2.7682 - accuracy: 0.1668
Epoch 5/2000
62/62 [==============================] - 1s 9ms/step - loss: 2.5421 - accuracy: 0.2286
Epoch 6/2000
62/62 [==============================] - 1s 9ms/step - loss: 2.3006 - accuracy: 0.2954
Epoch 7/2000
62/62 [==============================] - 1s 9ms/step - loss: 2.0550 - accuracy: 0.3464
Epoch 8/2000
62/62 [==============================] - 1s 10ms/step - loss: 1.8532 - accuracy: 0.4102
Epoch 9/2000
62/62 [==============================] - 1s 10ms/step - loss: 1.6817 - accuracy: 0.4423
Epoch 10/2000
62/62 [==============================] - 1s 10ms/step - loss: 1.5345 - accuracy: 0.4

In [41]:
model.save("model.hdf5")

c:\Users\Srinivas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
model = tf.keras.models.load_model("model.hdf5", compile = False)

In [43]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [44]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence = 0.5) as hands:
    

    while cap.isOpened():
        ret , frame = cap.read()
        
        
        putcurrent = []
        
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        image = cv2.flip(image,1)
        
        image.flags.writeable = False
        result = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
     
        
        if result.multi_hand_landmarks:
            for num,hand in enumerate(result.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(0,0,256) , thickness=2,circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(0,0,0) , thickness=3,circle_radius=2))
                
            for i in result.multi_hand_landmarks[0].landmark:
                putcurrent.append(i.x)
                putcurrent.append(i.y)
                putcurrent.append(i.z)
             
            
            nparr = np.array(putcurrent)
            
            predicted = list(model.predict(nparr.reshape(1,63))[0])
            
            fans = predicted.index(max(predicted))
        
            image = cv2.putText(image,"Predicted Value: "+ str(my_dict_rev[str(fans)]),(100,100),cv2.FONT_HERSHEY_PLAIN,2,(0,0,0),2)
            
        cv2.imshow('Pleiades was here' , image)
        
        if cv2.waitKey(10)  == ord('z'):
            break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 33ms/step


In [70]:
X_test.shape

(840, 63)

In [71]:
xtest = np.reshape(X_test,(840,63))
ytest = y_test[:]
xtest.shape,ytest.shape

((840, 63), (840,))

In [72]:
from sklearn import metrics

ypred = list(model.predict(xtest))

index = 0

for i in ypred:
    i = list(i)
    ypred[index] = i.index(max(i))
    index = index + 1

ypred

metrics.accuracy_score(ytest,ypred)

27/27 [==============================] - 0s 2ms/step


0.9940476190476191

In [73]:
from sklearn.metrics import precision_score, recall_score, f1_score, mean_squared_error

# Calculate predictions
ypred = list(model.predict(xtest))
index = 0
for i in ypred:
    i = list(i)
    ypred[index] = i.index(max(i))
    index += 1

# Calculate precision
precision = precision_score(ytest, ypred, average='weighted')

# Calculate recall
recall = recall_score(ytest, ypred, average='weighted')

# Calculate F1 score
f1 = f1_score(ytest, ypred, average='weighted')

# Calculate mean squared error
mse = mean_squared_error(ytest, ypred)

# Print the metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Mean Squared Error:", mse)

27/27 [==============================] - 0s 2ms/step
Precision: 0.9941968588464544
Recall: 0.9940476190476191
F1 Score: 0.99402510923376
Mean Squared Error: 0.30833333333333335
